### Simulating a protein with Molly.jl

In [ ]:
using Molly
### This is from the standard folder of Molly library.
data_dir = joinpath(dirname("/Users/leticiamadureira/.julia/packages/Molly/"), "PMLnv", "data")
#### First, we need to read in a force field in ```OpenMM XML format``` and read in a coordinate file in a format supported by ```Chemfiles.jl```.
#### Myelom protein
ff = OpenMMForceField(
    joinpath(data_dir, "force_fields", "ff99SBildn.xml"),
    joinpath(data_dir, "force_fields", "tip3p_standard.xml"),
    joinpath(data_dir, "force_fields", "his.xml"),
)
#### This sets up a system in the same data structures as above and that is simulated in the same way. Here we carry out an energy minimization, simulate with a ```Langevin integrator``` and use a ```StructureWriter``` to write the trajectory as a PDB file.
sys = System(
    joinpath(data_dir, "6mrr_equil.pdb"),
    ff;
    loggers=(
        energy=TotalEnergyLogger(10),
        writer=StructureWriter(10, "traj_6mrr_1ps.pdb", ["HOH"]),
    ),
)

minimizer = SteepestDescentMinimizer()
simulate!(sys, minimizer)

random_velocities!(sys, 298.0u"K")
simulator = Langevin(
    dt=0.001u"ps",
    temperature=300.0u"K",
    friction=1.0u"ps^-1",
)

simulate!(sys, simulator, 5_000; n_threads=Threads.nthreads())